---
# 1. Chronicle Authentication 🔒
To initiate an authenticated session in Chronicle, begin by uploading your keyfile.

In [ ]:
#@title Keyfile Upload
from os import rename
from google.colab import files

KEYFILE = 'chronicle.key'

print('Select and upload the rule data')
uploaded = files.upload()
rename(list(uploaded.keys())[0], f'{KEYFILE}')
print('🟢 Keyfile uploaded successfuly')

In [ ]:
#@title Session Start
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
from googleapiclient import _auth

def init_webclient(keyfile, region="North America"):
  SCOPES = [
      'https://www.googleapis.com/auth/chronicle-backstory',  # regular backstory API
      'https://www.googleapis.com/auth/malachite-ingestion',  # ingestion API
      'https://www.googleapis.com/auth/cloud-platform'        # dataplane API (experimenting)
  ]
  if region == 'North America': region_prefix = ''; cbn_region = 'US'; cli_region = 'US'
  elif region == 'Europe': region_prefix = 'europe-'; cbn_region = 'EUROPE'; cli_region = 'EUROPE'
  elif region == 'United Kingdom': region_prefix = 'europe-west2-'; cbn_region = 'EUROPE'; cli_region = 'EUROPE-WEST2'
  elif region == 'Asia (Singapore)': region_prefix = 'asia-southeast1-'; cbn_region = 'ASIA'; cli_region = 'ASIA-SOUTHEAST1'
  elif region == 'Australia (Sydney)': region_prefix = 'australia-southeast1-'; cbn_region = 'AUSTRALIA'; cli_region = 'AUSTRALIA-SOUTHEAST1'
  elif region == 'Tel Aviv': region_prefix = 'me-west1-'; cbn_region = 'AUSTRALIA'; cli_region = 'ME-WEST1'
  credentials = service_account.Credentials.from_service_account_file(keyfile, scopes=SCOPES)
  http_client = _auth.authorized_http(credentials)
  session = AuthorizedSession(credentials)
  return (http_client, session, region_prefix, cbn_region, cli_region)

http_client, session, region_prefix, cbn_region, cli_region = init_webclient(KEYFILE)
print('🟢 Webclient initialized')

---
# 2. Rules Export 📤
Once a session has been established, you have the capability to retrieve the rules and export the selected fields.

In [ ]:
#@title Load
from urllib.parse import urlencode
from http import HTTPStatus
from json import loads

def request(http_client, region_prefix, page_size=2000, page_token=''):
  url_params = {'page_size': page_size}
  if page_token:
    url_params['page_token'] = page_token
  uri = f'https://{region_prefix}backstory.googleapis.com/v2/detect/rules?{urlencode(url_params)}'
  res = http_client.request(uri, 'GET')
  if res[0].status == HTTPStatus.OK:
    return loads(res[1])
  else:
    return loads(res[1]).get('error').get('message')

in_rules = request(http_client, region_prefix)
print(f'🟢 {len(in_rules["rules"])} rules loaded from Chronicle')

In [ ]:
#@title Export
from csv import writer

RULES = 'chronicle-rules.csv'

out_rules = list()
header = ['name', 'description',
          'severity', 'priority', 'mitre_technique', 'detection_score',
          'reference', 'author', 'response',
          'id', 'type', 'status', 'last_update']

for rule in in_rules['rules']:
  try:
    out_rules.append([
        rule['ruleName'],
        rule['metadata']['description'],
        rule['metadata']['severity'],
        rule['metadata']['priority'],
        rule['metadata']['mitre_technique'],
        rule['metadata']['detection_score'],
        rule['metadata']['reference'],
        rule['metadata']['author'],
        rule['metadata']['response'],
        rule['ruleId'],
        rule['ruleType'],
        rule['liveRuleEnabled'],
        rule['versionCreateTime']
    ])
  except KeyError:
    print(f'🟡 {rule["ruleName"]}: not parsed, check the meta fields')

with open(f'{RULES}', 'w') as f:
  write = writer(f)
  write.writerow(header)
  write.writerows(out_rules)

print(f'🟢 {len(out_rules)} rules exported in {RULES}')
files.download(RULES)